### 데이터셋 다운로드
https:/github.com/bab2min/corpus.git

In [140]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [85]:
col = ['index', 'positive', 'content']
train = pd.read_csv('./data/naver_shopping.txt', delimiter='\t', header=None)
test = pd.read_csv('./data/steam.txt',delimiter='\t', header=None)
#read_csv 옵션에 header=None으로 읽기

In [86]:
pd.set_option("display.max_colwidth", 100) # 컬럼의 너비를 넓혀서 리뷰를 다 볼 수 있도록 세팅하는 것

display("Train data")
display(train.head())
display("Test data")
display(test.head())

'Train data'

,0,1
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요.
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전화했더니 바로주신다했지만 배송도 누락되어있었네요.. 확인안하고 바로 선물했으면 큰일날뻔했네...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


'Test data'

,0,1
0,0,노래가 너무 적음
1,0,"돌겠네 진짜. 황숙아, 어크 공장 그만 돌려라. 죽는다."
2,1,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지
3,1,차악!차악!!차악!!! 정말 이래서 왕국을 되찾을 수 있는거야??
4,1,시간 때우기에 좋음.. 도전과제는 50시간이면 다 깰 수 있어요


# 데이터 전처리

In [87]:
#인코딩
train[0] = train[0].replace(1,0)
train[0] = train[0].replace(2,0)
train[0] = train[0].replace(5,1)
train[0] = train[0].replace(4,1)

train[0]

0         1
1         0
2         1
3         0
4         1
         ..
199995    0
199996    1
199997    1
199998    1
199999    1
Name: 0, Length: 200000, dtype: int64

In [ ]:
#함수를 이용한 라벨인코딩(?)
def labeling(row):
    if row[0] >= 4:
        return 1
    else:
        return 0
    
train[0] = train.apply(labeling, axis=1)
test[0] = test.apply(labeling, axis=1)

In [99]:
text_train = train[1]
text_test = test[1]

In [89]:
#결측치 확인
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       200000 non-null  int64 
 1   1       200000 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       100000 non-null  int64 
 1   1       100000 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [97]:
Tf = TfidfVectorizer()
#단어사전 구축
Tf.fit(train[1])
display(Tf.vocabulary_)

{'배공빠르고': 144285,
 '택배가': 328253,
 '엉망이네용': 236510,
 '저희집': 281929,
 '밑에층에': 135641,
 '말도없이': 112901,
 '놔두고가고': 74168,
 '아주좋아요': 216524,
 '바지': 137872,
 '정말': 285979,
 '좋아서2개': 294839,
 '구매했어요': 42223,
 '이가격에': 256947,
 '대박입니다': 86272,
 '바느질이': 136348,
 '조금': 290456,
 '엉성하긴': 236600,
 '하지만': 346223,
 '편하고': 337222,
 '가성비': 20624,
 '최고예요': 316198,
 '선물용으로': 186039,
 '빨리': 168936,
 '받아서': 140909,
 '전달했어야': 283022,
 '하는': 344679,
 '상품이었는데': 180520,
 '머그컵만': 118326,
 '와서': 247036,
 '당황했습니다': 85699,
 '전화했더니': 284315,
 '바로주신다했지만': 137302,
 '배송도': 145136,
 '누락되어있었네요': 74473,
 '확인안하고': 357143,
 '바로': 137071,
 '선물했으면': 186370,
 '큰일날뻔했네요': 325909,
 '이렇게': 258757,
 '배송이': 146749,
 '오래걸렸으면': 243233,
 '사는거': 171745,
 '다시': 80389,
 '생각했을거같아요': 183841,
 '아쉽네요': 214925,
 '민트색상': 134677,
 '예뻐요': 241942,
 '손잡이는': 193034,
 '거는': 27533,
 '용도로도': 250269,
 '사용되네요': 173523,
 'ㅎㅎ': 16077,
 '비추합니다': 166540,
 '계란': 32762,
 '뒤집을': 96099,
 '완전': 247570,
 '불편해요': 162420,
 'ㅠㅠ': 17196,
 '코팅도': 323545,
 '묻어나고': 1302

## 형태소 분석기 함수
- morphs(text): 텍스트에서 형태소를 반환
- nouns(text): 텍스트에서 명사를 반환
- phrases(text): 텍스트에서 어절을 추출
- pos(text): 텍스트에서 품사정보를 부착하여 반환

In [ ]:
#형태소 분석기 사용
steam_morphs = [okt.morphs(doc) for doc in tqdm(steam_reviews[])]

In [154]:
okt = Okt()

In [164]:
def myTokenizer(text):
    res = okt.pos(text) #품사 부착
    df = pd.DataFrame(res, columns=["형태소", "품사"])
    #불리언색인
    df2 = df[(df['품사']=='Noun') | (df['품사']=='Adjective') | (df['품사']=='Verb')]
    return df2['형태소']

In [165]:
# tokenizer 파라미터 설정 (사용자 정의 함수와 연결해서 씀)
tfidf = TfidfVectorizer(tokenizer=myTokenizer) 

In [166]:
tfidf.fit(text_train)

C:\Users\AI\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<function myTokenizer at 0x000001FB310DF280>)

In [161]:
tfidf.vocabulary_

AttributeError: 'TfidfVectorizer' object has no attribute 'vocabulary_'

In [98]:
len(Tf.vocabulary_)

362867

In [124]:
X_train = Tf.transform(text_train)
y_train = train[0]

In [125]:
print(X_train.shape, y_train.shape)

(200000, 362867) (200000,)


In [136]:
#모델링
logi_model = LogisticRegression(n_jobs=-1)
#교차검증
rs = cross_val_score(logi_model, X_train, y_train, cv=3)
logi_model.fit(X_train,y_train) #교차 검증과 학습은 다른 것?

LogisticRegression(n_jobs=-1)

In [137]:
#예측해보기
sample_reviews = ["돈이 존나 아깝다",
                 "배송이 너무 빨라서 좋아요",
                 "생각보다 별로에요",
                 "선물용으로 샀는데 선물 받고 좋아해요"]

sample_tranform = Tf.transform(sample_reviews)  #정형화 작업
pre = logi_model.predict_proba(sample_tranform) #확률값 예측
pre

array([0, 1, 0, 1], dtype=int64)

In [138]:
#파이프라인 생성
pipeline = Pipeline([
    ('shopping_tfidf', TfidfVectorizer()),  #전처리 프로그램이기 때문에 평가를 못해서 튜닝을 할 수 없어서 파이프라인으로 붙여서 튜닝
    ('shopping_logi', LogisticRegression())
])


In [141]:
#그리드 서치 파라미터 설정
grid_param = {
    "shopping_tfidf__max_df" : [7000, 8500, 10000],        
    "shopping_tfidf__min_df" : [1, 5, 8, 10,],
    "shopping_tfidf__ngram_range" : [(1,2), (2,2), (1,3)],
    "shopping_logi__C" : [0.001, 0.01, 0.1, 10, 100, 1000]
}

In [142]:
grid = GridSearchCV(pipeline, grid_param, cv=3, n_jobs=-1) 
#grid.fit(text_train, y_train) 
#X_train을 넣지 않는 이유는 정형화까지 파이프로 묶여 있기 때문

C:\Users\AI\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('shopping_tfidf', TfidfVectorizer()),
                                       ('shopping_logi',
                                        LogisticRegression())]),
             n_jobs=-1,
             param_grid={'shopping_logi__C': [0.001, 0.01, 0.1, 10, 100, 1000],
                         'shopping_tfidf__max_df': [7000, 8500, 10000],
                         'shopping_tfidf__min_df': [1, 5, 8, 10],
                         'shopping_tfidf__ngram_range': [(1, 2), (2, 2),
                                                         (1, 3)]})

In [147]:
print("최고조합 :",grid.best_params_)
print("최고점수 :",grid.best_score_)
print("하이퍼 파라미터 수정전 :", rs.mean())

최고조합 : {'shopping_logi__C': 10, 'shopping_tfidf__max_df': 10000, 'shopping_tfidf__min_df': 1, 'shopping_tfidf__ngram_range': (1, 2)}
최고점수 : 0.8764800013188306
하이퍼 파라미터 수정전 : 0.8753399999687668


In [144]:
#최적의 파라미터로 학습된 모델
best_model=grid.best_estimator_

In [153]:
best_model.predict_proba(["아니 말 타고 길 가다가 옆사람 지나치려고 속력내서 건너가는데 옆에 있던 그 새끼가 먼저 총 들이밀고 시비걸었단 말이에요 난 죄 없다고아무튼 갓겜입니다 첫 말에게 막시무스춘배라고 이름도 지어줬어요 여기서 우마무스메 하고있습니다",
               "엔딩이후 그때까지 해왔던 노고와 아서와의 몰입감을 돌려줘...차라리 아서가 마이카 같은역이였으면 정떼기라도 쉬운데ㅜㅜ",
               "플스판 다깨고 피씨판 또깨는 중. 이게 진정한 오픈월드 장르고, 이거 한번 하고나면 다른 어줍잖은 게임들이 오픈월드랍시고 내놓은거 보면 토나옴. 사이버펑크 니네 말이야."])

array([[0.86797935, 0.13202065],
       [0.99337629, 0.00662371],
       [0.75908851, 0.24091149]])